In [1]:
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

import pandas as pd

In [2]:
# l'endettement n'est pas un critère très significatif

#influences MACRO, 2013, PIB récession ou inflation, croissance +/- fortes

#métaclasses par labels 4/2

#bfr gros: fragilité (ou gros besoin)

# working capital (bfr=Net Operatiing Capital) / SA


# ta= total assets

# ebitda/ta
# shareholder funds / TA
# *change in shareholder equitiy
# cash market + mark sec / TA
# *#EBIT/ TA
# Tot deb / Shareholder funds
# *Cash flow / Tot det
# *cash flow / current liabilities
# ebit/ tot sales
# cash flow / tot sales



col_names=[
    #Liquidity
           '(Cash+Mark.Sec)/Current Liabilities',#*
           '(Cash+Mark.Sec)/Total Sales',#*
           'Cash/Current Assets',
           'Cash/Total Assets',
           'Current Assets/Current Liabilities',
           'Current Assets/Total Assets',
           'Current Liabilities/Total Assets',
           'Current Liabilities/Total Sales',
           'Inventories/Total Assets',
           'Quick Assets/Current Liabilities',
           'Quick Assets/Total Assets',
           'Working Capital/Total Assets',
           'Working Capital/Total Sales',
    #Solvency
           'Financial Debt/Cash Flow',
           'Financial Expenses/EBITDA',
           'Financial Expenses/Net Income',
           'Financial Expenses/Total Assets',
           'Financial Expenses/Value Added',
    #Profitability
           'Cash Flow/Shareholder Funds',
           'Cash Flow/Total Assets',
           'EBIT/Shareholder Funds',
           'EBIT/Total Assets',#
           'EBITDA/Permanent Equity',
           'EBITDA/Total Assets',
           'Net Income/Shareholder Funds',
           'Net Income/Total Assets',
           'Profit before Tax/Shareholder Funds',
    #Financial Structure
           'Long Term Debt/Shareholder Funds',
           'Long Term Debt/Total Assets',
           'Net Op.Work.Capital/Total Assets',
           'Shareholder Funds/Permanent Equity',
           'Shareholder Funds/Total Assets',#
           'Total Debt/Shareholder Funds',#
           'Total Debt/Total Assets',
    #Activity
           'Cash Flow/Total Sales',#
           'Cash Flow/Value Added',
           'EBIT/Value Added',
           'EBITDA/Total Sales',#
           'Gross Trading Profit/Total Sales',
           'Net Income/Total Sales',
           'Net Income/Value Added',
           'Value Added/Fixed Assets',
           'Value Added/Total Assets',
           'Value Added/Total Sales',
    #Turnover
           'Accounts Payable/Total Sales',
           'Current Assets/Total Sales',
           'Inventories/Total Sales',
           'Net.Op.Work.Capital/Total Sales',
           'Receivables/Total Sales',
           'Total Sales/Total Assets',
    'Année']

X=np.genfromtxt('X_train.csv')#../input/
y=np.genfromtxt('y_train.csv')

X_test_final=np.genfromtxt('X_test.csv')

print('X shape:',X.shape)
print('X_test_final shape:',X_test_final.shape)

X shape: (8001, 51)
X_test_final shape: (4000, 51)


In [3]:
# attributs par ordre d'importance selon la random forest
FI=[32,  7, 47, 41, 26, 24, 38, 17, 35, 34, 23, 11, 28, 46, 18, 25, 43,
        6, 13, 44,  5, 39, 36, 42, 48,  8, 45, 37, 22, 29, 49, 27, 20, 21,
        1, 40, 19,  3,  0,  2, 30, 12, 31, 15,  4, 14, 10, 16, 33,  9]
for i in range(15):
    print(i+1,col_names[FI[i]])

1 Total Debt/Shareholder Funds
2 Current Liabilities/Total Sales
3 Net.Op.Work.Capital/Total Sales
4 Value Added/Fixed Assets
5 Profit before Tax/Shareholder Funds
6 Net Income/Shareholder Funds
7 Gross Trading Profit/Total Sales
8 Financial Expenses/Value Added
9 Cash Flow/Value Added
10 Cash Flow/Total Sales
11 EBITDA/Total Assets
12 Working Capital/Total Assets
13 Long Term Debt/Total Assets
14 Inventories/Total Sales
15 Cash Flow/Shareholder Funds


In [4]:
def normalizeByYr(X,X_test_final):
    X_2=np.copy(X)
    X_test_final_2=np.copy(X_test_final)
    X_means={}
    X_stds={}
    
    X_means[2011]=np.nanmean(X[X[:,-1]==2011], axis=0)
    X_stds[2011]=np.nanstd(X[X[:,-1]==2011],axis=0)
    X_means[2011][-1]=0;X_stds[2011][-1]=1

    X_means[2012]=np.nanmean(X[X[:,-1]==2012], axis=0)
    X_stds[2012]=np.nanstd(X[X[:,-1]==2012],axis=0)
    X_means[2012][-1]=0;X_stds[2012][-1]=1

    X_means[2013]=np.nanmean(X[X[:,-1]==2013], axis=0)
    X_stds[2013]=np.nanstd(X[X[:,-1]==2013],axis=0)
    X_means[2013][-1]=0;X_stds[2013][-1]=1

    def normalizeRow(x):
        return (x-X_means[x[-1]])/X_stds[x[-1]]

    X_2=np.apply_along_axis(normalizeRow, 1, X_2)

    X_means[2013]=np.nanmean(X_test_final[X_test_final[:,-1]==2013], axis=0)
    X_stds[2013]=np.nanstd(X_test_final[X_test_final[:,-1]==2013],axis=0)
    X_means[2013][-1]=0;X_stds[2013][-1]=1

    X_means[2014]=np.nanmean(X_test_final[X_test_final[:,-1]==2014], axis=0)
    X_stds[2014]=np.nanstd(X_test_final[X_test_final[:,-1]==2014],axis=0)
    X_means[2014][-1]=0;X_stds[2014][-1]=1

    X_test_final_2=np.apply_along_axis(normalizeRow, 1, X_test_final_2)
    return X_2,X_test_final_2

In [5]:
#X_2,X_test_final_2=normalizeByYr(X,X_test_final)
#X_2=X_2[~np.any(np.isnan(X_2), axis=1)]
#df['Label'] = df['Label'].apply(lambda x: 'Faillite' if x==1 else 'Saine')
X_2,X_test_final_2=np.copy(X),np.copy(X_test_final)

from sklearn.preprocessing import MinMaxScaler

df=pd.DataFrame(X_2,columns=col_names)
df_final=pd.DataFrame(X_test_final_2,columns=col_names)
df['Label']=y
df=df.dropna()
df_final=df_final.dropna()

def normalisationMinMax():
    y_nonan=df['Label']
    y_annee=df['Année']
    y_final_annee=df_final['Année']
    X_2=df.values[:,:-1]
    X_test_final_2=df_final.values
    X_minmax=np.hstack((MinMaxScaler().fit_transform(X_2[:,:-1]),y_annee.values.reshape(-1,1)))
    X_final_minmax=np.hstack((MinMaxScaler().fit_transform(X_test_final_2[:,:-1]),y_final_annee.values.reshape(-1,1)))
    df=pd.DataFrame(X_minmax,columns=col_names)
    df_final=pd.DataFrame(X_final_minmax,columns=col_names)
    df['Label']=y_nonan
    
df['Label'] = df['Label'].apply(lambda x: 'Faillite' if x==1 else 'Saine')


In [6]:
def smartPlot(annee,col1_name,col2_name,xlim,ylim,s_value):
    df_annee=df[df['Année']==annee]
    if annee>=2013:
        df_final_annee=df_final[df_final['Année']==annee]
    
    df_annee_saine=df_annee[df_annee['Label']=='Saine']
    df_annee_faillite=df_annee[df_annee['Label']=='Faillite']

    col1_saine=df_annee_saine[col1_name]
    col2_saine=df_annee_saine[col2_name]
    col1_faillite=df_annee_faillite[col1_name]
    col2_faillite=df_annee_faillite[col2_name]

    plt.figure(figsize=(20,20))
    if annee>=2013:
        plt.scatter(df_final_annee[col1_name],df_final_annee[col2_name],c='blue',label="Entreprise à déterminer",s=s_value)
    if annee<=2013:
        plt.scatter(col1_saine,col2_saine,c='g',label="Entreprise saine",s=s_value)
        plt.scatter(col1_faillite,col2_faillite,c='r',label="Entreprise en faillite",s=s_value)
    plt.xlabel(col1_name)
    plt.ylabel(col2_name)
    plt.xlim(-xlim,xlim)
    plt.ylim(-ylim,ylim)
    plt.legend()
    plt.show()

In [7]:
m = interactive(smartPlot,
                annee=widgets.RadioButtons(options=[2011,2012,2013,2014]),
                col1_name=col_names,
                col2_name=col_names,
                xlim=(1,3,1),
                ylim=(1,3,1),
                s_value=(1,50,5))

display(m)

#'(Cash+Mark.Sec)/Current Liabilities',#*
#'(Cash+Mark.Sec)/Total Sales',#*
#'EBIT/Total Assets',#
#'Shareholder Funds/Total Assets',#
#'Total Debt/Shareholder Funds',#
#'Cash Flow/Total Sales',#
#'EBITDA/Total Sales',#

interactive(children=(RadioButtons(description='annee', options=(2011, 2012, 2013, 2014), value=2011), Dropdow…

* 'Current Assets/Current Liabilities' indé année si > certain epsilon tout est sain à part qq outlier sur 2012.

In [8]:
#def distrib():
def plotDistrib(annee,col_name,xmin,xmax,bw_value):
    
    df_annee=df[df['Année']==annee]

    plt.figure(figsize=(20,20))
    
    if annee<=2013:
        df_annee_saine=df_annee[df_annee['Label']=='Saine']
        df_annee_faillite=df_annee[df_annee['Label']=='Faillite']
        col_saine=df_annee_saine[col_name]
        col_faillite=df_annee_faillite[col_name]
        col_saine=col_saine[col_saine>xmin];col_saine=col_saine[col_saine<xmax]
        col_faillite=col_faillite[col_faillite>xmin];col_faillite=col_faillite[col_faillite<xmax]
        col_saine.plot.kde(bw_value,color='g',label="Entreprises saines")
        col_faillite.plot.kde(bw_value,color='r',label="Entreprises en faillite")
        
        col_faillite.append(col_saine).plot.kde(bw_value,color='b',label="Toutes les entreprises d'entraînement")
    if annee>=2013:
        df_final_annee=df_final[df_final['Année']==annee]
        col_final=df_final_annee[col_name]
        col_final=col_final[col_final>xmin];col_final=col_final[col_final<xmax]
        col_final.plot.kde(bw_value,color='gray',label="Entreprises à déterminer")
    plt.xlim(xmin,xmax)
    plt.legend()
    plt.show()

In [9]:
m = interactive(plotDistrib,
                annee=widgets.RadioButtons(options=[2011,2012,2013,2014]),
                col_name=col_names,
                xmin=(-50,0,0.5),
                xmax=(0,50,0.5),
                bw_value=(0.01,1,0.05),
                continuous_update=False)

display(m)
# 1 Total Debt/Shareholder Funds
# 2 Current Liabilities/Total Sales
# 3 Net.Op.Work.Capital/Total Sales
# 4 Value Added/Fixed Assets
# 5 Profit before Tax/Shareholder Funds
# 6 Net Income/Shareholder Funds
# 7 Gross Trading Profit/Total Sales
# 8 Financial Expenses/Value Added
# 9 Cash Flow/Value Added
# 10 Cash Flow/Total Sales
# 11 EBITDA/Total Assets
# 12 Working Capital/Total Assets
# 13 Long Term Debt/Total Assets
# 14 Inventories/Total Sales
# 15 Cash Flow/Shareholder Funds

interactive(children=(RadioButtons(description='annee', options=(2011, 2012, 2013, 2014), value=2011), Dropdow…

In [10]:
if False:
    print(df_2011.iloc[0])
    print('-----')
    print(df_2011.iloc[1])
    print('-----')
    print(df_2011.iloc[2])
    print('-----')
    print(df_2011.iloc[3])
    print('-----')
    print(df_2011.iloc[4])
    print('-----')
    print(df_2011.iloc[5])
    print('-----')
    print(df_2011.iloc[6])